In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'/home/arch_aditya/Desktop/text-summarizer'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidation:
    root_dir : Path
    STATUS_FILE : str
    ALL_REQUIRED_FILES : list